In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# URL에서 html가져오는 함수
def HTML가져오기(url):
   _html = ""
   resp = requests.get(url) # URL에 GET 요청을 보내서 돌아온 응답(response)
   if resp.status_code == 200: # 200이면 정상적으로 응답이 왔다는 뜻
      _html = resp.text # 거기서 텍스트 즉 html만 가져오고
   return _html # 그걸 반환함


In [3]:
tables = soup.find('body').find_all('table', recursive=False)
print len(tables)
print tables[1].text.strip()

SyntaxError: invalid syntax (<ipython-input-3-5c98125daeee>, line 2)

In [83]:
#리뷰 가져오는 함수
def 리뷰_가져오기(URL):
    html = HTML가져오기(URL) # 위에서 정의된 함수
    soup = BeautifulSoup(html, 'html.parser') # beatifuleSoup 라이브러리에 html넣어서 파싱함 
    
    네이버140자평 = soup.find('div', {'class': 'score_result'}) # score_result 클래스를 가진 div 태그
    별점들 = 네이버140자평.find_all("div",{"class": "star_score"}) 
    리뷰들 = 네이버140자평.find_all("div",{"class": "score_reple"})
#    따봉들 = 네이버140자평.find_all("div",{"class": "btn_area"})
#    언따봉들 = 네이버140자평.findAll("div",{"class": "btn_area"})[1]
    
    별점 = [int(별점.em.text) for 별점 in 별점들] # [number + 10 for number in [1,2,3]]의 결과는 [11, 12, 13]
    리뷰 = [리뷰.p.text.replace('BEST', '') for 리뷰 in 리뷰들]
    따봉 = [int(따봉.strong.span.text) for 따봉 in 따봉들]
    언따봉 = [int(언따봉.strong.span.text) for 언따봉 in 언따봉들]

    결과 = list(zip(별점,리뷰,따봉,언따봉)) # list(zip([1,2,3], [4,5,6]))의 결과는 [(1,4), (2,5), (3,6)]
    
    return 결과

In [84]:

영화코드 = '136900'
URL = 'http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='+ 영화코드 +'&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
가져올_페이지_수 = 1

네이버리뷰 = []
for 페이지 in range(1,가져올_페이지_수+1):
    네이버리뷰 += 리뷰_가져오기(URL + str(페이지))
    time.sleep(1.3)
    
    
print(네이버리뷰)

AttributeError: 'NavigableString' object has no attribute 'strong'

## csv로 쓰기

In [5]:
import csv

csv파일 = open('네이버영화리뷰-샤잠.csv', 'w',  newline='') # 파일이 이미 있든 없든 쓰기 전용으로 열어서
csvout = csv.DictWriter(csv파일, ['별점', '리플']) # 헤더 만들고
csvout.writeheader() 

for 리뷰 in 네이버리뷰: 
    csvout.writerow({'별점': 리뷰[0], '리플': 리뷰[1]}) # 데이터 한 줄씩 써넣음

csv파일.close()